In [1]:
import sys
sys.path.insert(0, '/VIS_3TH/')

import numpy as np
from numba import njit

from setup import SHORT_PATH
import importlib.util
game_name = 'MachiKoro'

def add_game_to_syspath(game_name):
    if len(sys.argv) >= 2:
        sys.argv = [sys.argv[0]]
    sys.argv.append(game_name)

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHORT_PATH}Base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

add_game_to_syspath(game_name)
env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward
numba_main_2 = env.numba_main_2

In [2]:
@njit()
def player(state, per):
    validActions = getValidActions(state)
    validActions[52] = validActions[52]*state[18]*state[17]*state[16]
    validActions = np.where(validActions == 1)[0]

    if 1 in validActions: # Đổ xúc xắc
        return 1, per
    
    # if (0 in validActions) and (state[117] not in (2, 3, 4, 5)):
    #     return 0, per

    if (np.sum(state[16:20]==3)) and (validActions[52] == 0) and (validActions[49] == 0):
        if 53 in validActions:
            return 53, per
    
    if np.sum(state[1:16]) >= 8:
        for i in (51, 52, 50, 49):
            if i in validActions:
                return i, per
    
    for i in (35, 34, 36, 38, 39):
        if (i in validActions) and (state[i-33] <= 5):
            return i, per
    if (37 in validActions) and (state[37-33] <= 3):
        return 37, per
        
    if (state[39] + state[59] + state[79] >= 1):
        if (43 in validActions) and (state[43-33]<=2):
            return 43, per
        if (44 in validActions) and (state[44-33]<=1):
            return 44, per  
        if (42 in validActions) and (state[42-33]<=1):
            return 42, per
        
    if 53 in validActions:
        return 53, per
        
    action = validActions[np.random.randint(len(validActions))]
    return action, per

In [3]:
win, per = numba_main_2(player, 100000, np.array([]), 1)
win

38897